Vamos a pasar el .pkl a .csv para polars

In [1]:
import polars as pl
import pandas as pd

In [2]:
schema = {
    "session_id": pl.Int64,
    "date": pl.Utf8,               # String en Polars se representa como Utf8
    "timestamp_local": pl.Utf8,   # String -> Utf8
    "add_to_cart": pl.Int64,
    "user_id": pl.Float64,
    "country": pl.Int64,
    "partnumber": pl.Int32,
    "device_type": pl.Int64,
    "pagetype": pl.Float64
}

schema_clients = {
    "user_id": pl.Float64,
    "country": pl.Int64,
    "R": pl.Int64,
    "F": pl.Int64,
    "M": pl.Float64
}


In [3]:
train = pl.read_csv("../../data/raw/train.csv", schema=schema)
clients = pl.read_csv("../../data/raw/users_data.csv", schema=schema_clients)
products_pandas = pd.read_pickle("../../data/raw/products.pkl")
products = pl.from_pandas(products_pandas)

In [3]:
result_q1 = (
    products
    .filter((pl.col("color_id") == 3) & (pl.col("discount") == 1))
    .sort("family")
    .select("partnumber")
    .head(1)
)

print("Q1 Result:", result_q1)


Q1 Result: shape: (1, 1)
┌────────────┐
│ partnumber │
│ ---        │
│ i32        │
╞════════════╡
│ 17265      │
└────────────┘


In [4]:
# Filtrar usuarios con compras totales < 500 y contar cuántos hay en cada país
filtered_clients = (
    clients
    .filter(pl.col("M") < 500)  # Filtrar por compras < 500
    .group_by("country")  # Agrupar por país
    .agg(pl.len().alias("count"))  # Contar los usuarios por país
    .sort("count", descending=True)  # Ordenar por cantidad de usuarios
    .select('country')
    .head(1)
)

# País con más usuarios con compras < 500
top_country = filtered_clients["country"]

# Filtrar usuarios en el país top y ordenar por F, R, y user_id
result_q2 = (
    clients
    .filter(pl.col("country") == top_country)  # Filtrar por el país top
    .sort(["F", "R", "user_id"], descending=[False, True, False])  # Ordenar
    .select('user_id')  # Obtener el primer usuario
    .head(1)
)

print("Q2 Result:", result_q2)

Q2 Result: shape: (1, 1)
┌─────────┐
│ user_id │
│ ---     │
│ i64     │
╞═════════╡
│ 187374  │
└─────────┘


In [6]:
# result_q3 = (
#     train
#     .filter(pl.col("add_to_cart") == 1)  # Filtrar productos añadidos al carrito
#     .group_by("partnumber")
#     .agg([
#         pl.len().alias("total_visits"),
#         pl.sum("add_to_cart").alias("cart_adds"),
#     ])
#     #.with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
#     .select(pl.col("visits_per_cart_add").mean().round(2))
# )

result_q3_filtered= (
    train
    .group_by('partnumber')  # Agrupar por 'partnumber'
    .agg(pl.sum('add_to_cart').alias('cart_adds'))  # Sumar 'add_to_cart' para obtener 'cart_adds'
    .filter(pl.col('cart_adds') > 1)  # Filtrar solo los productos con 'cart_adds' > 1
)

result_q3 = (
    train
    .filter(pl.col('partnumber').is_in(result_q3_filtered['partnumber']))
    .group_by("partnumber")
    .agg([
        pl.len().alias("total_visits"),
        pl.sum("add_to_cart").alias("cart_adds"),
    ])
    .with_columns((pl.col("total_visits") / pl.col("cart_adds")).alias("visits_per_cart_add"))
    .select(pl.col("visits_per_cart_add").mean().round(2))
)

print(result_q3)



shape: (1, 1)
┌─────────────────────┐
│ visits_per_cart_add │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 21.9                │
└─────────────────────┘


In [7]:
products_mini = products.select(['partnumber', 'discount'])
train_concat = train.join(products_mini, on='partnumber', how='inner')

result_q4 = (
    train_concat
    .filter((pl.col("add_to_cart") == 1) & (pl.col("discount") == 1))
    .group_by("device_type")
    .agg(pl.len().alias("usage_count"))  # Contar el número de filas para cada dispositivo
    .sort("usage_count", descending=True)  # Ordenar por la frecuencia de uso
    .head(1)
)

print("Q4 Result:", result_q4)


Q4 Result: shape: (1, 2)
┌─────────────┬─────────────┐
│ device_type ┆ usage_count │
│ ---         ┆ ---         │
│ i64         ┆ u32         │
╞═════════════╪═════════════╡
│ 1           ┆ 169439      │
└─────────────┴─────────────┘


In [ ]:
# Paso 1: Filtrar los usuarios con las frecuencias de compra más altas dentro de su país
top_3_frequency_users = (
    clients
    .sort(["country", "F"], descending=[False, True])  # Ordenar por país y frecuencia de compra
    .group_by("country")
    .head(3)  # Seleccionar los 3 usuarios con mayor frecuencia dentro de su país
)
top_3_frequency_users

# Paso 2: Filtrar por `device_type == 3` y contar las interacciones con productos
result_q5 = (
    train  # Supongo que 'sessions' contiene las interacciones de los usuarios
    .filter(pl.col("user_id").is_in(top_3_frequency_users["user_id"]))  # Filtrar solo los usuarios del top 3
    .filter(pl.col("device_type") == 3)  # Filtrar por `device_type == 3`
    .group_by("user_id")  # Agrupar por usuario
    .agg(pl.n_unique("partnumber").alias("unique_interactions"))  # Contar productos distintos
    .sort("unique_interactions", descending=True)  # Ordenar por interacciones más altas
    .head(1)  # Seleccionar el usuario con más interacciones
)

# Resultado final
print("Q5 Result:", result_q5)


Q5 Result: shape: (1, 2)
┌─────────┬─────────────────────┐
│ user_id ┆ unique_interactions │
│ ---     ┆ ---                 │
│ f64     ┆ u32                 │
╞═════════╪═════════════════════╡
│ 72153.0 ┆ 81                  │
└─────────┴─────────────────────┘


In [ ]:
clients_mini = (
    clients
    .select([
        pl.col('user_id'),
        pl.col('country').alias('user_country')
    ])
)
products_mini = products.select(['partnumber', 'family'])
train_mini = train.select(['partnumber', 'country'])

clients_concat = clients_mini.join(train, on='user_id', how='inner')
clients_concat = clients_concat.join(products_mini, on='partnumber', how='inner')

In [5]:
result_q6 = (
    clients_concat
    .filter(pl.col("country") != pl.col("user_country"))  # Filtrar interacciones fuera del país del usuario
    .select("family")
    .n_unique()
)

print("Q6 Result:", result_q6)

Q6 Result: 116


In [ ]:
from datetime import datetime

# Filtrar interacciones de los primeros 7 días de junio
result_q7 = (
    train
    .filter(
        (pl.col("add_to_cart") == 1) &
        (pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d") >= datetime(2023, 6, 1)) &
        (pl.col("date").str.strptime(pl.Date, fmt="%Y-%m-%d") <= datetime(2023, 6, 7))
    )
    .groupby(["family", "page_type"])
    .count()
    .sort(["family", "count", "page_type"], descending=[False, True, False])
    .groupby("family")
    .agg(pl.first("page_type").alias("most_frequent_pagetype"))
    .to_dicts()
)

print("Q7 Result:", result_q7)